In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with project


In [5]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-train-churn")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the chrun dataset
print("Loading Data...")
churn = pd.read_csv('data/telco-csv.csv')

churn2 = churn.copy()

# drop those colume have null values
churn2 = churn2.drop(['loglong','logtoll','logequi','logcard','logwire','lninc'],axis = 1)

# convert categorical to num
for x in churn2.columns:
    if churn2[x].dtypes == 'object':
        churn2[x] = pd.Categorical(churn[x]).codes
        
# Separate features and labels
X, y = churn2[['region', 'tenure', 'age', 'marital', 'address', 'income', 'ed',
       'employ', 'retire', 'gender', 'reside', 'tollfree', 'equip', 'callcard',
       'wireless', 'longmon', 'tollmon', 'equipmon', 'cardmon', 'wiremon',
       'longten', 'tollten', 'equipten', 'cardten', 'wireten', 'multline',
       'voice', 'pager', 'internet', 'callid', 'callwait', 'forward', 'confer',
       'ebill',
       'custcat']].values, churn2['churn'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'chrun_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/chrun_model.pkl', model_name='chrun_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

Starting experiment: mslearn-train-churn
Loading Data...
Training a decision tree model
Accuracy: 0.6633333333333333
AUC: 0.5806878306878307
Model trained and registered.


In [6]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

chrun_model version: 1
	 Training context : Inline Training
	 AUC : 0.5806878306878307
	 Accuracy : 0.6633333333333333


churn_classifier version: 1


churn_model version: 1
	 Training context : Pipeline
	 AUC : 0.5969863668978714
	 Accuracy : 0.6666666666666666




In [7]:
model = ws.models['churn_model']
print(model.name, 'version', model.version)

churn_model version 1


In [8]:
import os

# Create a folder for the deployment files
deployment_folder = './churn_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_churn.py'
script_path = os.path.join(deployment_folder,script_file)

./churn_service folder created.


In [9]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'churn_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-churn', 'churn']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Writing ./churn_service/score_churn.py


In [10]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
service_env.inferencing_stack_version="latest"

inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "churn-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-03 16:42:02+00:00 Creating Container Registry if not exists.
2023-02-03 16:42:02+00:00 Registering the environment.
2023-02-03 16:42:04+00:00 Use the existing image.
2023-02-03 16:42:05+00:00 Submitting deployment to compute.
2023-02-03 16:42:11+00:00 Checking the status of deployment churn-service..
2023-02-03 16:44:26+00:00 Checking the status of inference endpoint churn-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [13]:
import json

# This time our input is an array of two feature arrays
x_new = [0,45,30,1,0,63,2,4,1,1,4,1,1,1,1,7.1,40.75,39.75,16.5,50.85,314.8,1849.5,1717.2,775,2239.65,1,1,1,1,1,1,1,1,1,2]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
print(predicted_classes[0])

Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '616', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Fri, 03 Feb 2023 16:56:38 GMT', 'Server': 'nginx', 'X-Ms-Client-Request-Id': '5499ae70-37b3-448c-9a3c-97e9d39c74f5', 'X-Ms-Request-Id': '5499ae70-37b3-448c-9a3c-97e9d39c74f5', 'X-Ms-Run-Function-Failed': 'True', 'X-Ms-Server-Version': 'azmlinfsrv/0.7.6', 'X-Request-Id': 'b8ac2afc-b42b-41cb-9bac-5eca7e0e6f23'}
Content: b'Expected 2D array, got 1D array instead:\narray=[0.00000e+00 4.50000e+01 3.00000e+01 1.00000e+00 0.00000e+00 6.30000e+01\n 2.00000e+00 4.00000e+00 1.00000e+00 1.00000e+00 4.00000e+00 1.00000e+00\n 1.00000e+00 1.00000e+00 1.00000e+00 7.10000e+00 4.07500e+01 3.97500e+01\n 1.65000e+01 5.08500e+01 3.14800e+02 1.84950e+03 1.71720e+03 7.75000e+02\n 2.23965e+03 1.00000e+00 1.00000e+00 1.00000e+00 1.00000e+00 1.00000e+00\n 1.00000

WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '616', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Fri, 03 Feb 2023 16:56:38 GMT', 'Server': 'nginx', 'X-Ms-Client-Request-Id': '5499ae70-37b3-448c-9a3c-97e9d39c74f5', 'X-Ms-Request-Id': '5499ae70-37b3-448c-9a3c-97e9d39c74f5', 'X-Ms-Run-Function-Failed': 'True', 'X-Ms-Server-Version': 'azmlinfsrv/0.7.6', 'X-Request-Id': 'b8ac2afc-b42b-41cb-9bac-5eca7e0e6f23'}
Content: b'Expected 2D array, got 1D array instead:\narray=[0.00000e+00 4.50000e+01 3.00000e+01 1.00000e+00 0.00000e+00 6.30000e+01\n 2.00000e+00 4.00000e+00 1.00000e+00 1.00000e+00 4.00000e+00 1.00000e+00\n 1.00000e+00 1.00000e+00 1.00000e+00 7.10000e+00 4.07500e+01 3.97500e+01\n 1.65000e+01 5.08500e+01 3.14800e+02 1.84950e+03 1.71720e+03 7.75000e+02\n 2.23965e+03 1.00000e+00 1.000